In [ ]:
import albumentations as A
import os
import random
import config
import sys
import cv2
import numpy as np
import shutil


sys.path.append('../util')
from DatasetAugmentation import *


#Enter the root name of the original dataset
original_dataset_path = './dataset/Foot Ulcer Segmentation Challenge'
output_base = config.dataset_path

/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Removing images without masks from dataset

In [2]:


def move_black_masks(
    masks_path, 
    imgs_path, 
    output_path, 
    extensions=(".png", ".jpg", ".jpeg", ".tif")
):
    os.makedirs(output_path, exist_ok=True)
    black_masks = []

    for root, _, files in os.walk(masks_path):
        for file in files:
            if file.lower().endswith(extensions):
                mask_path = os.path.join(root, file)
                img = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

                if img is None:
                    print(f"[WARNING] Unable to read {mask_path}")
                    continue

                # Convert to grayscale if you have multiple channels
                if len(img.shape) == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                # Checks if all pixels are 0
                if np.all(img == 0):
                    base_name, _ = os.path.splitext(file)
                    black_masks.append(base_name)

                    # Matching image path
                    img_path = None
                    for ext in extensions:
                        attempt = os.path.join(imgs_path, base_name + ext)
                        if os.path.exists(attempt):
                            img_path = attempt
                            break

                    # Move mask
                    new_name_mask = f"{base_name}_mask.png"
                    output_mask = os.path.join(output_path, new_name_mask)
                    shutil.move(mask_path, output_mask)

                    # Move image if exists
                    if img_path and os.path.exists(img_path):
                        destino_img = os.path.join(output_path, os.path.basename(img_path))
                        shutil.move(img_path, destino_img)
                    else:
                        print(f"[WARNING] Matching image not found for {file}")

                    print(f"[OK] Moved: {file} and corresponding image")

    print(f"\nTotal black masks moved:{len(black_masks)}")
    return black_masks


# ======== Usage example ========

# Path of the original masks
dir_mascaras = f"{original_dataset_path}/train/labels"
# Original images path
dir_imagens = f"{original_dataset_path}/train/images"
# Destination path where they will be moved
dir_destino = f"{original_dataset_path}/train-fix/"

move_black_masks(dir_mascaras, dir_imagens, dir_destino)


[OK] Moved: 0092.png and corresponding image
[OK] Moved: 0131.png and corresponding image
[OK] Moved: 0223.png and corresponding image
[OK] Moved: 0281.png and corresponding image
[OK] Moved: 0335.png and corresponding image
[OK] Moved: 0361.png and corresponding image
[OK] Moved: 0457.png and corresponding image
[OK] Moved: 0502.png and corresponding image
[OK] Moved: 0564.png and corresponding image
[OK] Moved: 0600.png and corresponding image
[OK] Moved: 0701.png and corresponding image
[OK] Moved: 0725.png and corresponding image
[OK] Moved: 0736.png and corresponding image
[OK] Moved: 0778.png and corresponding image
[OK] Moved: 0832.png and corresponding image
[OK] Moved: 0899.png and corresponding image
[OK] Moved: 0912.png and corresponding image
[OK] Moved: 0931.png and corresponding image
[OK] Moved: 0950.png and corresponding image

Total black masks moved:19


['0092',
 '0131',
 '0223',
 '0281',
 '0335',
 '0361',
 '0457',
 '0502',
 '0564',
 '0600',
 '0701',
 '0725',
 '0736',
 '0778',
 '0832',
 '0899',
 '0912',
 '0931',
 '0950']

### Moving 100 images from validation to test

In [3]:
from_img_dir = os.path.join(original_dataset_path, "validation", "images")
from_lbl_dir = os.path.join(original_dataset_path, "validation", "labels")
to_img_dir = os.path.join(original_dataset_path, "test", "images")
to_lbl_dir = os.path.join(original_dataset_path, "test", "labels")

os.makedirs(to_img_dir, exist_ok=True)
os.makedirs(to_lbl_dir, exist_ok=True)

images = os.listdir(from_img_dir)
selected = random.sample(images, 100)

for img in selected:
    shutil.move(os.path.join(from_img_dir, img), os.path.join(to_img_dir, img))
    shutil.move(os.path.join(from_lbl_dir, img), os.path.join(to_lbl_dir, img))

### Augmentation

In [4]:

# -----------------------------
# Parameters
# -----------------------------
N = 10  # number of augmentations
num_to_valid = 0    # number of images to move from train to valid
num_to_test  = 0    # number of images to move from train to test



target_size  = (224, 224)
random.seed(42)

# -----------------------------
# Entry and exit paths
# -----------------------------
orig_train_img_dir  = os.path.join(original_dataset_path, 'train/images')
orig_train_mask_dir = os.path.join(original_dataset_path, 'train/labels')
orig_valid_img_dir  = os.path.join(original_dataset_path, 'validation/images')
orig_valid_mask_dir = os.path.join(original_dataset_path, 'validation/labels')
orig_test_img_dir   = os.path.join(original_dataset_path, 'test/images')
orig_test_mask_dir  = os.path.join(original_dataset_path, 'test/labels')


output_dirs = {
    'train_images': os.path.join(output_base, 'images/train'),
    'train_labels': os.path.join(output_base, 'labels/train'),
    'valid_images': os.path.join(output_base, 'images/valid'),
    'valid_labels': os.path.join(output_base, 'labels/valid'),
    'test_images':  os.path.join(output_base, 'images/test'),
    'test_labels':  os.path.join(output_base, 'labels/test'),
}

transforms = A.Compose([
    A.Resize(*target_size, interpolation=cv2.INTER_NEAREST),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7, border_mode=cv2.BORDER_REFLECT),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
])

augment_dataset(N, num_to_valid, num_to_test,
                    orig_train_img_dir, orig_train_mask_dir,
                    orig_valid_img_dir, orig_valid_mask_dir,
                    orig_test_img_dir, orig_test_mask_dir,
                    output_base,
                    transforms,
                    )

Total images in the original dataset: 791
→ Training: 791
→ Validation (of training): 0
→ Test (training): 0

With N=1, total images generated in training will be: 1582


/home/alan/miniconda3/envs/pytorch5070/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Copiando test: 100%|██████████| 300/300 [00:00<00:00, 556.49it/s]


→ 100 images copied from the original valid folder.
→ 100 images copied from the original test folder.


Enlarging workout images: 100%|██████████| 791/791 [00:12<00:00, 61.71it/s]


Final summary:
train_images: 1582 files
train_labels: 1582 files
valid_images: 100 files
valid_labels: 100 files
test_images: 100 files
test_labels: 100 files
